Tworzenie samej bazy danych

In [ ]:
CREATE DATABASE Projekt_BD

Tworzenie wszystkich tabel w bazie danych

In [ ]:
-- Tworzenie bazy danych


-- Usuwanie wszystkich tabel w razie potrzeby
-- DROP TABLE naprawy;
-- DROP TABLE pracownicy;
-- DROP TABLE Adresy;
-- DROP TABLE Historia_pracownikow;
-- DROP TABLE Klienci;
-- DROP TABLE Kontakty;
-- DROP TABLE Urzadzenia;
-- DROP TABLE Urzadzenia_klientow;


-- Tabela Pracownicy
CREATE TABLE Pracownicy (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nazwisko VARCHAR(255) NOT NULL,
	imie VARCHAR(255) NOT NULL
);


-- Tabela Klienci
CREATE TABLE Klienci (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nazwa_firmy VARCHAR(255) NOT NULL
);

-- Tabela osób do kontaktu z klientami
CREATE TABLE Kontakty (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_firmy INTEGER REFERENCES Klienci(id),
	nazwisko VARCHAR(255) NOT NULL,
	imie VARCHAR(255) NOT NULL,
	tel_kierunkowy VARCHAR(3),
	telefon INTEGER,
	email VARCHAR(255)
);

-- Tabela adresów klientów
CREATE TABLE Adresy (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_klienta INTEGER REFERENCES Klienci(id),
	ulica VARCHAR(255) NOT NULL,
	numer_budynku VARCHAR(15) NOT NULL,
	numer_lokalu VARCHAR(15),
	miasto VARCHAR(255) NOT NULL,
	kod_pocztowy VARCHAR(15)
);

-- Tabela Urzadzenia
CREATE TABLE Urzadzenia (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    producent VARCHAR(255) NOT NULL,
    model VARCHAR(255) NOT NULL
);

-- Tabela z urzadzeniami klientów
CREATE TABLE Urzadzenia_klientow (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_modelu INTEGER REFERENCES Urzadzenia(id),
	numer_seryjny VARCHAR(255) NOT NULL
);

-- Tabela Naprawy
CREATE TABLE Naprawy (
    id INTEGER IDENTITY(1,1) PRIMARY KEY,
    nr_naprawy INTEGER NOT NULL,
    pracownik_id INTEGER REFERENCES Pracownicy(id),
    klient_id INTEGER REFERENCES Klienci(id),
    urzadzenie_id INTEGER REFERENCES Urzadzenia(id),
    opis_klienta TEXT,
    status VARCHAR(50) NOT NULL,
    diagnoza TEXT,
    wykonane_czynnosci TEXT,
    czesci_do_wymiany TEXT,
    data_przyjecia DATE NOT NULL,
    data_rozpoczecia DATE,
    data_zakonczenia DATE
);

-- Tabela Historia pracowników
-- Tabela rejestrująca wszsystkich pracownikow biorących udział w naprawie
CREATE TABLE Historia_pracownikow (
	id INTEGER IDENTITY(1,1) PRIMARY KEY,
	id_naprawy INTEGER REFERENCES Naprawy(id),
	id_pracownika INTEGER REFERENCES Pracownicy(id)
);

-- 1. Zbieranie wszystkich napraw
SELECT * FROM Naprawy;

-- 2. Śledzenie ilości napraw w miesiącu
SELECT COUNT(*), strftime('%Y-%m', data_przyjecia) as Miesiac
FROM Naprawy
GROUP BY Miesiac;

-- 3. Ilość napraw dokonywanych przez każdego technika indywidualnie
SELECT pracownik_id, nazwisko, COUNT(*) as liczba_napraw
FROM Naprawy
JOIN Pracownicy ON Pracownicy.id = Naprawy.pracownik_id
GROUP BY pracownik_id;

-- 4. Wyszukiwanie napraw po kliencie i/lub numerze naprawy
-- Zmienne: @nazwa_firmy, @nr_naprawy
SELECT * FROM Naprawy
JOIN Klienci ON Klienci.id = Naprawy.klient_id
WHERE (nazwa_firmy = @nazwa_firmy OR @nazwa_firmy IS NULL)
AND (nr_naprawy = @nr_naprawy OR @nr_naprawy IS NULL);

-- 5. Sprawdzenie urządzeń, których producenta przychodzi najwięcej
SELECT producent, COUNT(*) as liczba_napraw
FROM Naprawy
JOIN Urzadzenia ON Urzadzenia.id = Naprawy.urzadzenie_id
GROUP BY producent
ORDER BY liczba_napraw DESC;

-- 6. Naprawy oczekujące po przyjęciu oraz w trakcie naprawy
SELECT * FROM Naprawy
WHERE status IN ('urządzenie przyjęte', 'w trakcie naprawy')
ORDER BY data_przyjecia ASC, data_rozpoczecia ASC;


Uzupełnienie tabel danymi

- Dodaję kilku przykładowych pracowników i tabelę urządzeń z której będzie można wykonać autouzupełnianie w aplikacji z GUI\*
- Reszta danych jest całkowicie przykładowa

\*Aplikacja z GUI poza aktualnym projektem

In [ ]:
-- Wprowadzanie danych Pracowników
INSERT INTO Pracownicy
VALUES ('Banasik', 'Gaweł'), 
	('Siekierski', 'Paweł'),
	('Zasada', 'Mariusz'),
	('Golan', 'Bolesław'),
	('Kłopotowski', 'Paweł'),
	('Cupryn', 'Andrzej'),
	('Andrzejewski', 'Dawid');

Select * FROM Pracownicy;

-- Wprowadzanie danych urządzeń
INSERT INTO Urzadzenia	VALUES
	('Zebra', 'ZT410'),
	('Zebra', 'ZT230'),
	('Zebra', 'GX430t'),
	('Zebra', 'ZD420'),
	('Zebra', 'ZD620'),
	('Zebra', 'ZXP Series 7'),
	('Zebra', 'ZXP Series 9'),
	('Zebra', 'ZQ620'),
	('Zebra', 'ZQ630'),
	('Zebra', 'ZQ520'),
	('Toshiba', 'B-EX4T1'),
	('Toshiba', 'B-FV4T'),
	('Toshiba', 'B-EV4T'),
	('Toshiba', 'B-SA4TM'),
	('Toshiba', 'B-EX4T2'),
	('Toshiba', 'B-SX5T'),
	('Toshiba', 'B-852-R'),
	('Toshiba', 'B-EP2DL'),
	('Toshiba', 'B-EV4D'),
	('Toshiba', 'B-FV4D'),
	('Honeywell', 'PC42t'),
	('Honeywell', 'PM43'),
	('Honeywell', 'PX940'),
	('Honeywell', 'PM23c'),
	('Honeywell', 'PX4i'),
	('Honeywell', 'PD43'),
	('Honeywell', 'PM42'),
	('Honeywell', 'PC23d'),
	('Honeywell', 'PX6i'),
	('Honeywell', 'PC42d'),
	('TSC', 'TTP-244 Pro'),
	('TSC', 'TA210'),
	('TSC', 'TE200'),
	('TSC', 'TX200'),
	('TSC', 'TTP-247'),
	('TSC', 'TC200'),
	('TSC', 'ME240'),
	('TSC', 'MX240'),
	('TSC', 'TTP-225'),
	('TSC', 'TTP-2410MT');


-- Klienci (Customers) - Sample Data
INSERT INTO Klienci (nazwa_firmy)
VALUES ('Bosch'), ('L''Oréal'), ('Example Company 1'), ('Example Company 2'), ('Example Company 3');

-- Kontakty (Contacts) - Sample Data
INSERT INTO Kontakty (id_firmy, nazwisko, imie, tel_kierunkowy, telefon, email)
VALUES
    (1, 'Contact1_LastName', 'Contact1_FirstName', '123', 456789, 'contact1@example.com'),
    (2, 'Contact2_LastName', 'Contact2_FirstName', '456', 987654, 'contact2@example.com'),
    (3, 'Contact3_LastName', 'Contact3_FirstName', '789', 123456, 'contact3@example.com'),
    (4, 'Contact4_LastName', 'Contact4_FirstName', '111', 222333, 'contact4@example.com'),
    (5, 'Contact5_LastName', 'Contact5_FirstName', '444', 555666, 'contact5@example.com');

-- Adresy (Addresses) - Sample Data
INSERT INTO Adresy (id_klienta, ulica, numer_budynku, numer_lokalu, miasto, kod_pocztowy)
VALUES
    (1, 'Street1', '123', 'A1', 'City1', '12345'),
    (2, 'Street2', '456', 'B2', 'City2', '67890'),
    (3, 'Street3', '789', 'C3', 'City3', '13579'),
    (4, 'Street4', '111', 'D4', 'City4', '24680'),
    (5, 'Street5', '222', 'E5', 'City5', '86420');

-- Urzadzenia_klientow (Customers' Devices) - Sample Data
INSERT INTO Urzadzenia_klientow (id_modelu, numer_seryjny)
VALUES
    (1, 'Serial1'),
    (2, 'Serial2'),
    (3, 'Serial3'),
    (4, 'Serial4'),
    (5, 'Serial5');

-- Naprawy (Repairs) - Sample Data (continued)
INSERT INTO Naprawy (nr_naprawy, pracownik_id, klient_id, urzadzenie_id, opis_klienta, status, diagnoza, wykonane_czynnosci, czesci_do_wymiany, data_przyjecia, data_rozpoczecia, data_zakonczenia)
VALUES
	(1, 1, 1, 1, 'Description1', 'urządzenie przyjęte', 'Diagnosis1', 'Actions1', 'Parts1', '2023-01-01', '2023-01-02', '2023-01-03'),
    (2, 1, 2, 2, 'Description2', 'w trakcie naprawy', 'Diagnosis2', 'Actions2', 'Parts2', '2023-02-01', '2023-02-02', '2023-02-03'),
    (3, 2, 3, 3, 'Description3', 'w trakcie naprawy', 'Diagnosis3', 'Acition3', 'Parts3', '2023-03-01', '2023-03-02', '2023-03-03'),
    (4, 2, 4, 4, 'Description4', 'urządzenie przyjęte', 'Diagnosis4', 'Actions4', 'Parts4', '2023-03-01', '2023-03-02', '2023-03-03'),
    (5, 3, 5, 5, 'Description5', 'w trakcie naprawy', 'Diagnosis5', 'Actions5', 'Parts5', '2023-04-01', '2023-04-02', '2023-04-03');

-- Historia_pracownikow (Employees' History) - Sample Data
INSERT INTO Historia_pracownikow (id_naprawy, id_pracownika)
VALUES
    (1, 1),
    (2, 1),
    (3, 2),
    (4, 2),
    (5, 3);


Lista procedur potrzebnych do obsługi bazy dancyh:

- Pokaz\_naprawy - Wyświetla listę ostatnich napraw
- Edytuj\_naprawe - Edycja wybranej naprawy
- Dodaj\_naprawe - Tworzenie nowego wpisu do listy napraw
- Usun\_naprawe - Usuwanie wpisu z tabeli naprawy
- Dodaj\_pracownika - Dodawanie nowego pracownika do bazy danych
- Usun\_pracownika - Usuwanie pracownika z bazy
- Dodaj\_klienta - Dodawanie nowego klienta
- Dodaj\_kontakt - Dodwanie osoby przypisanej do firmy (przedstawiciel danej firmy)
- Dodaj\_adres - Dodawanie adresu do klienta (jeden klient może posiadać wiele adresów)
- Edytuj\_klienta - Modyfikacja danych klienta
- Edytuj\_adres - Modyfikacja adresu klienta
- Edytuj\_kontakt - Modifikacja danych przedstawiciela
- Usun\_klienta - Usuwa klienta z bazy danych
- Usuń\_adres - Usuwa adres klienta
- Usuń\_kontakt - Usuwanie przedstawiciela
- Stat\_prac - Pokazuje ilosc napraw w jakiej dany pracownik brał udział w danym przedziale czasowym
- Stat\_klient - Pokazuje ilość napraw zleconych przez danego klieta
- Pokaz\_klient - Wyświelta listę danego klienta